In [54]:
import pandas as pd
import pandas_ta as ta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import requests
import matplotlib.pyplot as plt
from math import floor
from termcolor import colored as cl
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)
from binance.client import Client
import datetime as dt
import json
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from sklearn.metrics import classification_report

url = 'https://api.binance.com/api/v3/klines'
symbol = 'ADAUSDT'
interval = '1d'
start = str(int(dt.datetime(2022,3,1).timestamp()*1000))
end = str(int(dt.datetime(2022,11,1).timestamp()*1000))
par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end}
data = pd.DataFrame(json.loads(requests.get(url, params= par).text))
#format columns name
data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
data=data.astype(float)

df = data

def get_macd(price, slow, fast, smooth):
        exp1 = price.ewm(span = fast, adjust = False).mean()
        exp2 = price.ewm(span = slow, adjust = False).mean()
        macd = pd.DataFrame(exp1 - exp2).rename(columns = {'close':'macd'})
        signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
        hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
        return macd, signal, hist

data['macd'] = get_macd(data['close'], 26, 12, 9)[0]
data['macd_signal'] = get_macd(data['close'], 26, 12, 9)[1]
data['macd_hist'] = get_macd(data['close'], 26, 12, 9)[2]
data = data.dropna()

length = 14
data.ta.rsi(close='close', length=length, append=True, signal_indicators=True, xa=50, xb=55)

data["EMA10"] = ta.ema(data["close"], length=10)
data["EMA30"] = ta.ema(data["close"], length=30)
data['ATR'] = df.ta.atr(df['high'].values.any(), df['low'].values.any(), df['close'].values.any(), timeperiod=14)
#data['ADX'] = df.ta.adx(df['high'].values.any(), df['low'].values.any(), df['close'].values.any(), timeperiod=14)

data['ClgtEMA10'] = np.where(data['close'] > data['EMA10'], 1, -1)
data['EMA10gtEMA30'] = np.where(data['EMA10'] > data['EMA30'], 1, -1)
data['MACDSIGgtMACD'] = np.where(data['macd_signal'] > data['macd'], 1, -1)

data['Return'] = data['close'].pct_change(1).shift(-1)
data['target_cls'] = np.where(data.Return > 0, 1, 0)
data['target_rgs'] = data['Return']

data = data.dropna()

predictors_list = ['ATR','RSI_14', 'ClgtEMA10', 'EMA10gtEMA30', 'MACDSIGgtMACD']
X = data[predictors_list]

y_cls = data.target_cls
y_rgs = data.target_rgs

y=y_cls
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(X, y, test_size=0.3, random_state=432, stratify=y)

train_length = int(len(data)*0.70)
X_rgs_train = X[:train_length]
X_rgs_test = X[train_length:]
y_rgs_train = y_rgs[:train_length]
y_rgs_test = y_rgs[train_length:]

clf = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=6)
clf = clf.fit(X_cls_train, y_cls_train)

dot_data = tree.export_graphviz(clf, out_file=None,filled=True,feature_names=predictors_list)
graphviz.Source(dot_data)

y_cls_pred = clf.predict(X_cls_test)
report = classification_report(y_cls_test, y_cls_pred)

print(report)
print(y_cls_pred)
graphviz.Source(dot_data).view()



              precision    recall  f1-score   support

           0       0.56      0.94      0.70        34
           1       0.75      0.19      0.31        31

    accuracy                           0.58        65
   macro avg       0.66      0.57      0.51        65
weighted avg       0.65      0.58      0.51        65

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]


'Source.gv.pdf'